In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import cv2
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf

In [2]:
ROWS = 64
COLS = 64
CHANNELS = 3
CLASSES = 2

In [3]:
#read image dataset 
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

# data preparation
def init_data(image_datasets):
    m = len(image_datasets)
    X = np.zeros((m, ROWS, COLS, CHANNELS), dtype=np.uint8)
    y = np.zeros((1, m), dtype=np.uint8)
    for i, image_file in enumerate(image_datasets):
        X[i,:] = read_image(image_file)
        if 'Truck' in image_file.lower():
            y[0, i] = 0
        elif 'Horse' in image_file.lower():
            y[0, i] = 1   
    return X, y


# conversion to One Hot form
def OH(Y, C): 
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [4]:
train_images = ['Train_data/'+i for i in os.listdir('Train_data/')]
test_images =  ['Test_data/'+i for i in os.listdir('Test_data/')]

train_set_x, train_set_y = init_data(train_images)
test_set_x, test_set_y = init_data(test_images)

X_train = train_set_x/255
X_test = test_set_x/255

Y_train = OH(train_set_y, CLASSES).T
Y_test =  OH(test_set_y, CLASSES).T

print ("number of training examples =", X_train.shape[0])
print ("number of test examples =", X_test.shape[0])
print ("X_train shape:", X_train.shape)
print ("Y_train shape:", Y_train.shape)
print ("X_test shape:", X_test.shape)
print ("Y_test shape:", Y_test.shape)

number of training examples = 399
number of test examples = 198
X_train shape: (399, 64, 64, 3)
Y_train shape: (399, 2)
X_test shape: (198, 64, 64, 3)
Y_test shape: (198, 2)


In [5]:
def id_block(X, ks, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'


    Shortcut_data = X # Save the input value

    X = Conv2D(filters[0], kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters[1], kernel_size = (ks, ks), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters[2], kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut as an additional trace of data
    X = Add()([X, Shortcut_data])
    X = Activation('relu')(X)

    return X


def conv_block(X, ks, filters, stage, block, s = 2):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    #F1, F2, F3 = filters
    Shortcut_data = X # input value copy (... + X)

 
    X = Conv2D(filters[0], (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters[1], kernel_size=(ks, ks), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters[2], kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    Shortcut_data = Conv2D(filters[2], (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(Shortcut_data)
    Shortcut_data = BatchNormalization(axis = 3, name = bn_name_base + '1')(Shortcut_data)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, Shortcut_data])
    X = Activation('relu')(X)
    
    return X

In [6]:
#######################################################################
###                         ResNet-50                               ###
#######################################################################

In [7]:
def ResNet50(input_shape = (64, 64, 3), classes = 2):   

    # kernel_size
    ks = 3
    
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    
    # conv, batch norm, ReLu, Max Pool
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'Conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'batchnorm_1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    # conv, id block x 2 
    X = conv_block(X, ks, filters = [64, 64, 256], stage = 2, block='1', s = 1)
    X = id_block(X, 3, [64, 64, 256], stage=2, block='2')
    X = id_block(X, 3, [64, 64, 256], stage=2, block='3')
    # conv, id block x 3 
    X = conv_block(X, ks, filters = [128, 128, 512], stage = 3, block='1', s = 2)
    X = id_block(X, 3, [128, 128, 512], stage=3, block='2')
    X = id_block(X, 3, [128, 128, 512], stage=3, block='3')
    X = id_block(X, 3, [128, 128, 512], stage=3, block='4')
    # conv, id block x 5 
    X = conv_block(X, ks, filters = [256, 256, 1024], stage = 4, block='1', s = 2)
    X = id_block(X, 3, [256, 256, 1024], stage=4, block='2')
    X = id_block(X, 3, [256, 256, 1024], stage=4, block='3')
    X = id_block(X, 3, [256, 256, 1024], stage=4, block='4')
    X = id_block(X, 3, [256, 256, 1024], stage=4, block='5')
    X = id_block(X, 3, [256, 256, 1024], stage=4, block='6')
    # conv, id block x 2 
    X = conv_block(X, ks, filters = [512, 512, 2048], stage = 5, block='1', s = 2)
    X = id_block(X, 3, [512, 512, 2048], stage=5, block='2')
    X = id_block(X, 3, [512, 512, 2048], stage=5, block='3')
    # AVG pool, Flatten, FC
    X = AveragePooling2D((2, 2), name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet-50')

    return model

In [9]:
EPOCHS = 5
BATCH_SIZE = 64

model = ResNet50(input_shape = (ROWS, COLS, CHANNELS), classes = CLASSES)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs = EPOCHS, batch_size = BATCH_SIZE)

Epoch 1/5
7/7 [==============================] - 10s 1s/step - loss: 0.5877 - accuracy: 0.8521
Epoch 2/5
7/7 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000


In [10]:
prediction_test = model.evaluate(X_test, Y_test)
print ("Losses: " + str(prediction_test[0]))
print ("Accuracy: " + str(prediction_test[1]))

7/7 [==============================] - 1s 110ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Losses: 0.0
Accuracy: 1.0


In [11]:
model.summary()
model.save('ResNet50_test.h5')

Model: "ResNet-50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batchnorm_1 (BatchNormalization (None, 32, 32, 64)   256         Conv1[0][0]                      
__________________________________________________________________________________________